# Classification of the CIFAR10 Dataset

Each label is encoded, 0-9, representing a class. Each image is a 3072 length array. The images are 32x32, so the first 1024 values correspond to the red channel, followed by the green and the red.

In [1]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from skimage import io, transform
from torchvision import utils, transforms
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
data = unpickle('cifar-10-batches-py/data_batch_1')

In [5]:
x = data[b'labels']
y = data[b'data']

# Image Preprocessing Functions

In [6]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        
        h, w = image.shape[:2]
        
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
               
        return {'image': img, 'label': label}
    

class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size
            
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        
        top = np.random.randint(0, h-new_h)
        left = np.random.randint(0, w-new_w)
        
        image = image[top: top+new_h,
                      left: left+new_w]
        
        return {'image': image, 'label': label}
    
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        
        # swap color axis because
        # numpy iamge: HxWxC
        # torch image: CxHxW
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'label': label}

# Create the Dataset Object

In [7]:
class CIFARDataset(Dataset):
    def __init__(self, batch_lst:list, transform=None):
        """To be used when iterating over the 
        files within the root directory"""
        self.labels, self.images = [], []
        
        for file in batch_lst:
            data = unpickle(file)
            self.labels += data[b'labels']
            self.images += [cifar10_to_rgb(img) for img in data[b'data']]
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx:int) -> dict:
        """For any index in the range of img_lst
        it maps image to the image values
        and species to the type of bird"""
        sample = {'image': self.images[idx], 
                  'label': self.labels[idx]}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [8]:
def cifar10_to_rgb(arr_lst):
    ___ = []
    for k in range(32):
        __ = []
        for j in range(32):
            _ = []
            for i in range(3):
                _ += [arr_lst[1024*i+32*j+k]]
            __ += [_]
        ___ += [__]
    return np.array(___)

In [9]:
batch_lst = """cifar-10-batches-py/data_batch_1
               cifar-10-batches-py/data_batch_2
               cifar-10-batches-py/data_batch_3
               cifar-10-batches-py/data_batch_4
               cifar-10-batches-py/data_batch_5
            """.split()

CIFAR = CIFARDataset(batch_lst,
                     transform=transforms.Compose([
#                                       Rescale(256),
                                      RandomCrop(30),
                                      ToTensor()
                                  ]))

In [10]:
train_dataset, test_dataset = train_test_split(CIFAR, random_state=88)

# Use the Data Loader to Implement Batches

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=4,
                              shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=4,
                             shuffle=True, num_workers=4)

# Train The Model

### Define The Network

In [55]:
class Network(nn.Module):
    def __init__(self, dropout_rate=0):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 6, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 12, 6, padding=1)
        self.fc1 = nn.Linear(12*5*5, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 10)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))        
        x = x.view(-1, 12*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
    
net = Network()

### Create a Loss and Optimizer

In [56]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, 
                      momentum=0.9)#, weight_decay=0.00001)

### Train For Real

In [57]:
def train(num_epocs):
    losses = []
    for epoch in range(num_epocs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs = data['image'].float()
            labels = data['label']
            
            optimizer.zero_grad()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            losses += [loss.item()]
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 1000 == 999:
                print(f'[{epoch+1}, {i+1}] loss: {running_loss/1000}')
                running_loss = 0.0
        test()

                
def test():
    losses = []
    net.eval()
    for i, data in enumerate(test_dataloader, 0):
        inputs = data['image'].float()
        labels = data['label']
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses += [loss.item()]
    print(f'Test Loss: {np.mean(losses)}')

In [58]:
train(10)

[1, 1000] loss: 2.2939008927345275
[1, 2000] loss: 2.1241402662992477
[1, 3000] loss: 2.032047041296959
[1, 4000] loss: 1.9511365410089492
[1, 5000] loss: 1.8825261654257774
[1, 6000] loss: 1.850749166548252
[1, 7000] loss: 1.8128227937221526
[1, 8000] loss: 1.7780043205618858
[1, 9000] loss: 1.7489470357894898
Test Loss: 1.7463831238365173
[2, 1000] loss: 1.7063379893898964
[2, 2000] loss: 1.6882118046879768
[2, 3000] loss: 1.686789938688278
[2, 4000] loss: 1.6517485678791999
[2, 5000] loss: 1.6780622035861015
[2, 6000] loss: 1.675162107884884
[2, 7000] loss: 1.6032831273674966
[2, 8000] loss: 1.6278938120007516
[2, 9000] loss: 1.6340268864035605
Test Loss: 1.6140663200187684
[3, 1000] loss: 1.5913055840432644
[3, 2000] loss: 1.6013480325341225
[3, 3000] loss: 1.558751403450966
[3, 4000] loss: 1.564589849948883
[3, 5000] loss: 1.5612137846648693
[3, 6000] loss: 1.6119517208337784
[3, 7000] loss: 1.555200392127037
[3, 8000] loss: 1.5496837377548218
[3, 9000] loss: 1.546994085609913
Tes

In [59]:
optimizer = optim.SGD(net.parameters(), lr=0.000001,
                      momentum=0.9, weight_decay=0.00001)
train(3)

[1, 1000] loss: 1.2213223802745343
[1, 2000] loss: 1.1805383367836475
[1, 3000] loss: 1.139053843408823
[1, 4000] loss: 1.1851782874763013
[1, 5000] loss: 1.1985605453252792
[1, 6000] loss: 1.169822973549366
[1, 7000] loss: 1.191998456954956
[1, 8000] loss: 1.1771617861986161
[1, 9000] loss: 1.1685627602934838
Test Loss: 1.3114385506248474
[2, 1000] loss: 1.1707515446543693
[2, 2000] loss: 1.1656533801555633
[2, 3000] loss: 1.1393925215303897
[2, 4000] loss: 1.1863460129499435
[2, 5000] loss: 1.1807792509794235
[2, 6000] loss: 1.1562031549811362
[2, 7000] loss: 1.1528634119033814
[2, 8000] loss: 1.1906743482649327
[2, 9000] loss: 1.1841574289798737
Test Loss: 1.3104818570041656
[3, 1000] loss: 1.1668889275491237
[3, 2000] loss: 1.1750831956267356
[3, 3000] loss: 1.1272694622576236
[3, 4000] loss: 1.1672408270537853
[3, 5000] loss: 1.1675383879840373
[3, 6000] loss: 1.151933901399374
[3, 7000] loss: 1.154692899554968
[3, 8000] loss: 1.1852463311254977
[3, 9000] loss: 1.1885173217356204


In [60]:
torch.save(net.state_dict(), 'models/CIFAR10.pt')

In [61]:
train(3)

[1, 1000] loss: 1.1734067938923836
[1, 2000] loss: 1.1356912103891372
[1, 3000] loss: 1.1721566995382309
[1, 4000] loss: 1.1517045343816281
[1, 5000] loss: 1.1564928091168403
[1, 6000] loss: 1.153905338048935
[1, 7000] loss: 1.1609659651219846
[1, 8000] loss: 1.156905490398407
[1, 9000] loss: 1.1751214535832406
Test Loss: 1.3026150169849395
[2, 1000] loss: 1.1549373888373375
[2, 2000] loss: 1.1627117763459682
[2, 3000] loss: 1.1570463382303715
[2, 4000] loss: 1.1412937308847904
[2, 5000] loss: 1.1585473252534866
[2, 6000] loss: 1.157166733443737
[2, 7000] loss: 1.1884651707708835
[2, 8000] loss: 1.1400272831618785
[2, 9000] loss: 1.1451120183765888
Test Loss: 1.3047532446193695
[3, 1000] loss: 1.1618755494356154
[3, 2000] loss: 1.1068300998210907
[3, 3000] loss: 1.1816344597041606
[3, 4000] loss: 1.1490236032307148
[3, 5000] loss: 1.148387181609869
[3, 6000] loss: 1.1568145793676377
[3, 7000] loss: 1.133525220811367
[3, 8000] loss: 1.1476807802915574
[3, 9000] loss: 1.1606893203258515


In [62]:
train(5)

[1, 1000] loss: 1.1612608671784401
[1, 2000] loss: 1.1143279196321965
[1, 3000] loss: 1.1386787248849868
[1, 4000] loss: 1.1371249536573886
[1, 5000] loss: 1.1682247643768788
[1, 6000] loss: 1.1708780645132064
[1, 7000] loss: 1.1531031126379967
[1, 8000] loss: 1.1541125074028968
[1, 9000] loss: 1.129371852427721
Test Loss: 1.301788368244171
[2, 1000] loss: 1.1239619280695916
[2, 2000] loss: 1.1447209930717945
[2, 3000] loss: 1.1432888792455196
[2, 4000] loss: 1.1287364637553692
[2, 5000] loss: 1.1373072642087936
[2, 6000] loss: 1.150818851441145
[2, 7000] loss: 1.188343494385481
[2, 8000] loss: 1.128650383144617
[2, 9000] loss: 1.1448564236462115
Test Loss: 1.3067093565654755
[3, 1000] loss: 1.1244019352793693
[3, 2000] loss: 1.1273026388287544
[3, 3000] loss: 1.1394866808652877
[3, 4000] loss: 1.1009647597074508
[3, 5000] loss: 1.1343373363614082
[3, 6000] loss: 1.15392905575037
[3, 7000] loss: 1.1436365436315536
[3, 8000] loss: 1.1543157762289047
[3, 9000] loss: 1.1501931378245354
Te

In [63]:
torch.save(net.state_dict(), 'models/CIFAR10.pt')